In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andreyzotov","key":"ad18b354caa39bfbe97b7cf4020a4c48"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
# we need to set permissions 
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [0]:
!cp '/content/drive/My Drive/kaggle/bengali/input/train.csv' ./

In [0]:
cp '/content/drive/My Drive/kaggle/bengali/input/grapheme-imgs-128x128.zip' ./

In [0]:
!mkdir 128

In [0]:
!unzip

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [0]:
!unzip grapheme-imgs-128x128.zip -d 128

In [0]:
!cp '/content/drive/My Drive/kaggle/bengali/input/m_dense_eps25_sr025.h5' ./

In [9]:
!ls

128    grapheme-imgs-128x128.zip  m_dense_eps25_sr025.h5  train.csv
drive  kaggle.json		  sample_data


In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import  recall_score

from tqdm.auto import tqdm
import cv2

In [0]:
model = load_model('m_dense_eps25_sr025.h5')

In [0]:
train_np = pd.read_csv('train.csv').to_numpy()
train_np[:,0]+='.png'
train_m = train_np.shape[0]
train_df = pd.DataFrame(train_np)
train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
train_dir = '128'

In [0]:
def test_batch_generator(frame, train_dir,batch_size=64):    
    
    num_imgs = len(frame)
    stats = (0.0692, 0.2051)
    
    for batch_start in range(0, num_imgs,batch_size):   
            cur_batch_size = min(num_imgs,batch_start+batch_size)-batch_start

            idx = np.arange(batch_start,batch_start+cur_batch_size)
            names_batch = frame.iloc[idx,0].values
            imgs_batch = np.zeros((cur_batch_size,128,128,3))
            
            for j in range(cur_batch_size):
                img = cv2.imread(train_dir+'/'+names_batch[j])
                img = (img.astype(np.float32)/255.0 - stats[0])/stats[1]
                imgs_batch[j,] = img

            yield imgs_batch

In [8]:
batch_size=1024
test_gen = test_batch_generator(train_df,train_dir, batch_size=batch_size)
root_predicts,vowel_predicts, consonant_predicts = [],[],[]

for batch_x in tqdm(test_gen):
  batch_predict = model.predict(batch_x)
  for j in range(batch_predict[0].shape[0]):
    root_predicts += [np.argmax(batch_predict[0][j],axis=0)]
    vowel_predicts += [np.argmax(batch_predict[1][j],axis=0)]
    consonant_predicts += [np.argmax(batch_predict[2][j],axis=0)]

In [0]:
p_root=np.array(root_predicts[:]).reshape(-1)
p_vowel = np.array(vowel_predicts[:]).reshape(-1)
p_consonant = np.array(consonant_predicts[:]).reshape(-1)
l = len(p_root)
t_root=np.array(train_df.iloc[:l,1].values, dtype=int)
t_vowel=np.array(train_df.iloc[:l,2].values, dtype=int)
t_consonant=np.array(train_df.iloc[:l,3].values, dtype=int)

In [12]:
0.5*recall_score(t_root, p_root, average='macro')+0.25*recall_score(t_vowel, p_vowel, average='macro')+0.25*recall_score(t_consonant,p_consonant,average='macro')

0.9906922255616476